In [1]:
#  Games Played Linear Regression.ipynb - Baseball demonstration of linar regression.
#     Copyright (C) 2020  Geoffrey G. Messier
# 
#     This program is free software: you can redistribute it and/or modify
#     it under the terms of the GNU General Public License as published by
#     the Free Software Foundation, either version 3 of the License, or
#     (at your option) any later version.
# 
#     This program is distributed in the hope that it will be useful,
#     but WITHOUT ANY WARRANTY; without even the implied warranty of
#     MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#     GNU General Public License for more details.
# 
#     You should have received a copy of the GNU General Public License
#     along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [2]:
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy as sci
import scipy.special as scisp
import scipy.stats as scist
import datetime, copy, imp
import re
import sys
import numexpr as ne
import random


import MySQLdb

import pymysql.cursors;

from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D

tqdm.pandas()
plt.ion()

In [3]:
players = pd.read_hdf('~/data/baseball/PlayerSummary-2018.hdf',key='Data')
players

,Position,GamesPlayed,AtBats,AvgOuts,AvgOnBase,AvgRuns,AvgRbis
PlayerId,,,,,,,
abrej003,FirstBase,129,549,7.139535,0.327869,0.123862,0.078324
acunr001,LeftField,111,484,1.657658,0.367769,0.161157,0.070248
adaml001,LeftField,24,28,0.375000,0.357143,0.357143,0.142857
adamm002,FirstBase,117,332,3.658120,0.313253,0.126506,0.108434
adamw002,ShortStop,85,320,2.882353,0.350000,0.134375,0.059375
...,...,...,...,...,...,...,...
zimmb001,CenterField,36,114,2.277778,0.280702,0.122807,0.052632
zimmj003,Pitcher,2,2,0.000000,0.000000,0.000000,0.000000
zimmr001,FirstBase,85,318,6.541176,0.342767,0.103774,0.103774


In [4]:
nPlayers = len(players.index)
pct70 = players.GamesPlayed.sort_values().iloc[int(0.3*nPlayers)]
players70 = players[players.GamesPlayed >= pct70]

## Theory and Notation

- Let the $M \times (P+1)$ input data matrix $\mathbb{X} = [\ \mathbb{X}_0\ \mathbb{X}_1\ \ldots\ \mathbb{X}_p\ ]$, where $M$ is the number of training samples, $P$ is the number of data features and $\mathbb{X}_0$ is a column of 1's.
- This makes the linear hypothesis function equal to $\hat{\mathbb{Y}} = \mathbb{X}\mathbb{\beta}$.
- The sum of squares function and its derivatives:
\begin{equation*}
\begin{array}{c}
J(\beta) = (\mathbb{Y} - \mathbb{X}\beta)^T(\mathbb{Y} - \mathbb{X}\beta) \\
\delta J/ \delta\beta = - 2\mathbb{X}^T(\mathbb{Y} - \mathbb{X}\beta) \\
\delta^2 J/ (\delta\beta\delta\beta^T) = 2\mathbb{X}^T \mathbb{X} \\
\end{array}
\end{equation*}



## Direct Least Squares Coefficient Calculation
- The least square solution for the coefficients is found by setting the first derivative of $J(\beta)$ to zero and solving for $\beta$.  The result is $\hat{\beta} = \left(\mathbb{X}^T\mathbb{X}\right)^{-1}\mathbb{X}^T\mathbb{Y}$.
- If this calculation is too expensive, iterative methods like gradient descent can be used to solve for the model coefficients.

In [5]:
m = len(players70.index)
p = 4

# Target variable
y = np.array([ players70.GamesPlayed ]).transpose()

# Input variables (p=4 features)
x = np.ones((m,p+1))
x[:,1:] = players70[['AvgOuts','AvgOnBase','AvgRuns','AvgRbis']].to_numpy()
#x[:,1:] = players70[['AvgOuts','AvgOnBase']].to_numpy()
#x[:,1:] = players70[['AtBats']].to_numpy()

In [6]:
nrm = np.ones((1,p+1))
nrm[0,1:] = np.sqrt(x[:,1:].var(axis=0))
x = x/nrm

In [7]:
# Direct least squares calculation
betaLs = np.linalg.inv(x.T @ x) @ x.T @ y
yHat = x @ betaLs
betaLs

array([[-19.98833285],
       [ 14.15212167],
       [ 23.91125286],
       [ -0.78502377],
       [  2.51856959]])

In [8]:
# Z-score calculation
# - Scores greater than ~2 are significant at the 5% level.
varYHat = (y-yHat).var()
vs = np.linalg.inv( x.T @ x ).diagonal()
zScoreLs = np.abs( betaLs.T / np.sqrt( varYHat * vs ) )
degFreedom = m - p - 1
pVals = 1-scist.t.cdf(zScoreLs,degFreedom)

print('VarYHat: ', varYHat )
print('Z-Scores: ', zScoreLs )
print('P-Values: ', pVals )

VarYHat:  1449.8866523323609
Z-Scores:  [[ 4.09089264  8.69304716 11.55411656  0.46225475  1.46177651]]
P-Values:  [[2.40150055e-05 0.00000000e+00 0.00000000e+00 3.22021973e-01
  7.21282056e-02]]


These results suggest that we should be able to drop the average number of runs and RBIs without significantly affecting model accuracy.  More thoughtful techniques for adjusting/removing coefficients include subset discovery, Lasso and LAR.

## Higher Order Models
- Use the second order polynomial of each of the native data features.

In [9]:
m = len(players70.index)
p = 8

# Target variable
y = np.array([ players70.GamesPlayed ]).transpose()

# Input variables (p=4 features)
x = np.ones((m,p+1))
x[:,1:5] = players70[['AvgOuts','AvgOnBase','AvgRuns','AvgRbis']].to_numpy()
x[:,5:] = x[:,1:5]**2

In [10]:
nrm = np.ones((1,p+1))
nrm[0,1:] = np.sqrt(x[:,1:].var(axis=0))
x = x/nrm

In [11]:
# Direct least squares calculation
betaLs = np.linalg.inv(x.T @ x) @ x.T @ y
yHat = x @ betaLs
betaLs

array([[  8.44442507],
       [ 21.54661882],
       [-16.61195531],
       [  3.68803744],
       [ 32.25878367],
       [ -9.52877319],
       [ 33.15456024],
       [ -3.38022912],
       [-28.82767128]])

In [12]:
# Z-score calculation
# - Scores greater than ~2 are significant at the 5% level.
varYHat = (y-yHat).var()
vs = np.linalg.inv( x.T @ x ).diagonal()
zScoreLs = np.abs( betaLs.T / np.sqrt( varYHat * vs ) )
degFreedom = m - p - 1
pVals = 1-scist.t.cdf(zScoreLs,degFreedom)

print('VarYHat: ', varYHat )
print('Z-Scores: ', zScoreLs )
print('P-Values: ', pVals )

VarYHat:  1274.8341142447775
Z-Scores:  [[1.10232615 5.54740004 2.63215836 1.18709085 7.75458762 2.65936355
  5.61213515 1.27364884 7.68630026]]
P-Values:  [[1.35352680e-01 2.06921361e-08 4.33735447e-03 1.17800748e-01
  1.59872116e-14 4.00551816e-03 1.44939580e-08 1.01609006e-01
  2.62012634e-14]]


It appears that we now have a large number of statistically significant model parameters.  However, when correlation exists between features in the data, one feature is sometimes assigned a large positive coefficient and the other a large negative coefficient.  This occurs between lower an higher order terms on the same native data feature.  These parameters can sometimes cancel each other out without significantly affecting performance.  Ridge regression attempts to counter-act this by placing a restriction on the size of coefficients.

## Training, Overfitting and Generalization Error

- Training error is the sum of squares error between the model and the data used to train it.
- Generalization error is the difference between the model and data outside of the training set.
- As complexity is added, training error tends to go down.  However, there is a risk of the model overfitting the training set and resulting in an increase in generalization error.
- We can balance this by *cross-validation* which trains on a subset of the data and evaluates that model performance based on generalization error calculated by the data outside of the training set.

In [13]:
cvFrac = 0.3
nIter = 20

trainErr = np.zeros(nIter)
genErr = np.zeros(nIter)

In [14]:
m = len(players70.index)
p = 4
y = np.array([ players70.GamesPlayed ]).transpose()
x = np.ones((m,p+1))
x[:,1:] = players70[['AvgOuts','AvgOnBase','AvgRuns','AvgRbis']].to_numpy()

nrm = np.ones((1,p+1))
nrm[0,1:] = np.sqrt(x[:,1:].var(axis=0))
x = x/nrm

In [15]:
mTrain = int(m*(1-cvFrac))
xTrain = np.zeros((mTrain,p+1))
xCv = np.zeros((m-mTrain,p+1))
yTrain = np.zeros((mTrain,1))
yCv = np.zeros((m-mTrain,1))

varTrainAvg = 0
varCvAvg = 0

for iter in range(0,nIter):
    
    indTrain = random.sample(range(m),mTrain)
    indCv = [ i for i in range(m) if i not in indTrain ]
    
    xTrain[:,:] = x[indTrain,:]
    yTrain[:] = y[indTrain]
    xCv[:,:] = x[indCv,:]
    yCv[:] = y[indCv]
    
    beta = np.linalg.inv(xTrain.T @ xTrain) @ xTrain.T @ yTrain
    
    yHatTrain = xTrain @ beta
    yHatCv = xCv @ beta
    
    varTrain = (yTrain - yHatTrain).var()
    varCv = (yCv - yHatCv).var()
    
    print('Iteration %d - Train Var: %g, CV Var: %g' % (iter,varTrain,varCv))
        
    varTrainAvg += varTrain
    varCvAvg += varCv
    
print('----')
print('Avg Train Var: %g, Avg CV Var: %g' % (varTrainAvg/nIter,varCvAvg/nIter))
    





Iteration 0 - Train Var: 1436.61, CV Var: 1479.87
Iteration 1 - Train Var: 1414.32, CV Var: 1563.22
Iteration 2 - Train Var: 1432.63, CV Var: 1482.54
Iteration 3 - Train Var: 1405.83, CV Var: 1578.9
Iteration 4 - Train Var: 1426.9, CV Var: 1527.1
Iteration 5 - Train Var: 1469.21, CV Var: 1412.81
Iteration 6 - Train Var: 1451.38, CV Var: 1456.46
Iteration 7 - Train Var: 1428.14, CV Var: 1505.67
Iteration 8 - Train Var: 1469.05, CV Var: 1483.31
Iteration 9 - Train Var: 1410.15, CV Var: 1558.23
Iteration 10 - Train Var: 1353.06, CV Var: 1678.77
Iteration 11 - Train Var: 1457.86, CV Var: 1444.35
Iteration 12 - Train Var: 1394.07, CV Var: 1580.23
Iteration 13 - Train Var: 1455.19, CV Var: 1448.25
Iteration 14 - Train Var: 1429.94, CV Var: 1495.09
Iteration 15 - Train Var: 1462.51, CV Var: 1422.57
Iteration 16 - Train Var: 1386.18, CV Var: 1609.47
Iteration 17 - Train Var: 1440.18, CV Var: 1499.24
Iteration 18 - Train Var: 1409.58, CV Var: 1625.01
Iteration 19 - Train Var: 1442.05, CV Var: 1

In this case, our training and generalization error is quite close.  In the contrived examples of overfitting, we typically see a picture of a small number of data points and a model that curves to meet each point.  In our baseball data, we have a large number independent data samples and a smaller number of features so this overfitting isn't really possible.  We might see this more if a large portion of the players were clumped into a small number of highly correlated groups.

## Notes

- This section includes some notes related to Chapter 3 of [Elements of Statistical Learning](https://web.stanford.edu/~hastie/Papers/ESLII.pdf).

#### Introduction
- The training data can be represented by a $\mathbb{X} \in \mathcal{R}^{M\times P}$ matrix where $M$ is the number of training samples and $P$ is the number of features in the data.  The $m$th row is $\mathbb{x}_m = [\ x_{m1},\ldots x_{mP}\ ]$ and the $p$th column is $\mathbb{X}_p$.
- A note on the features:
  - We have $N \leq P$ actual features in the data.
  - The remaining features can be created by taking higher order *basis expansions* of the original features.
  - If we have qualitative data (ie. player position), we can include this data by "dummy coding" it where each value is represented by a different feature and each feature is equal to $\{0,1\}$ depending on if it is true or false.
  
- The hypothesis function is $h(\mathbb{x}_m) = \mathbb{\beta} \mathbb{x_m}^T$ where $\mathbb{\beta} = [\ \beta_0\ \ldots\ \beta_P\ ]$.
- The residual sum of squares, RSS$(\mathbb{\beta})$, is the squared difference between the hypothesis function output and the actual target variable values summed over the $M$ training samples.
- We can solve for the parameters that minimize RSS directly using the pseudo-inverse (least squares projection) of the entire $\mathbb{X}$ matrix.  This works even if $\mathbb{X}$ isn't full rank (some of the columns are linearly dependent on others).
- The matrix inverse can be avoided by minimizing RSS using gradient descent or Newton's method.

#### Data Distribution & Model Statistics
- Assume that $h(\mathbb{X})$ is the true model for the mean of the target variable data $\mathbb{Y}$ and that any deviation about that mean observed in the data is Gaussian distributed so that $\mathbb{Y} = h(\mathbb{X}) + \epsilon$ and $\epsilon$ is a Gaussian random variable.
- The estimated model parameters are Gaussian distributed and the estimated target variable variance is Chi squared distributed.

- These assumptions allow us to evaluate confidence in the model parameters.
- The model parameters can be normalized by their standard deviations to create a Z-score to for use in a statistical confidence calculation:
  - A good discussion of statistical inference and the null hypothesis can be found in Chapter 7 of [Campbell, et.al.](https://ucalgary-primo.hosted.exlibrisgroup.com/permalink/f/12m0b6n/01UCALG_ALMA51675036160004336).
  - The *null hypothesis* is often used since it's easier to disprove.  For example, a hypothesis could be "All Asian children have black hair." which is hard to prove since all Asian children would need to be observed.  However, the hypothesis is much easier to disprove.  The null hypothesis is "There is at least one Asian child who does not have black hair." which is easier to prove since there is a chance we could prove this by observing just one child.
  - Our hypothesis is that a data feature has influence over the target variable and its corresponding parameter is non-zero in order to reflect that influence.  The null hypothesis is that the parameter is equal to zero.
  - No parameter will be truly equal to zero and our task is to determine whether a non-zero value of $\beta$ is simply due to random variation.  
  - A $p$-value corresponds to the probability that we discard a statistically significant model parameter as null.  Typically, $p < 0.05$.  We can use the t-distribution to determine the threshold below which random variations in a zero mean parameter would fall $1-p$ percent of the time.  
  - If the Z-score for our parameter is larger than this threshold, we assume that the null hypothesis is false, that we can have confidence that our model parameter is statistically significant and that it should be retained in our model.
- The F-statistic can be used to determine whether a group of parameters can be dropped from the model.  Useful for when a categorical variable is represented by a group of dummy indicator function variables.
- 95% confidence intervals for each model parameter can also be calculated either individually or as a group (individual is probably best).

#### Gauss-Markov Theory
- Start by assuming a linear combination of the parameters $\mathbb{x}_0^T\mathbb{\beta}$.  In this case, the data vector $\mathbb{x}_0$ is fixed.  Our calculation of the linear combination $\hat{\mathbb{\theta}} = \mathbb{x}_0^T\hat{\mathbb{\beta}}$ is an approximation since we're working with estimated values of the real underlying parameters.
- These parameter estimates are generated from $\mathbb{X}$, which we also assume is fixed.
- The model is unbiased since the mean of the estimate equals the true values.  The theorem says the pseudo-inverse solution to the estimated parameters gives the minimum variance unbiased estimator.

- While the theorem says least squares has the minimum MSE of all unbiased estimators, there may be another estimator with lower MSE that *has bias*.
- Bias can be caused by shrinking or zeroing out least squares coefficients.

#### Subset Selection
- This is where we shrink parameters and/or remove variables to improve prediction accuracy (lowering prediction error by introducing bias) and make the model easier to understand by focusing on the most important variables.
- Subset selection retains only a subset of the variables.
- Easy to understand but there are criticisms of this method as being too chunky and/or relying on inaccurate statistics.

#### Shrinkage Methods
- Adds an extra penalty in the minimization to reduce the sum of squares of the model coefficients.  Shrinks the coefficients towards zero and each other.
- Tends to eliminate large negative coefficients balanced by large positive coefficients which tends to occur when input variables are correlated.
- Inputs need to be standardized (normalize to unit variance) and centered (means subtracted off).  The y-intercept parameter ($\beta_0$) is not included in the penalty and is estimated by setting it equal to the mean of the target variable.
- The model parameters can again be solved directly using the least squares solution (and likely gradient descent or Newton's method).
- An SVD analysis demonstrates that ridge regression tends to shrink components that do not correspond to a large amount of variance in the target variable.  Likely because there is not a lot of statistical correlation.
- Lasso has the nice feature that it keeps some variables right at zero but requires a quadratic programming solution.
- LAR is a simpler approach that results in almost the same performance as Lasso.